In [1]:
import requests
import wptools
import re
# from bs4 import BeautifulSoup
import pandas as pd

In [2]:
page = wptools.page('Lebanese_Civil_War')
page.get_parse()

en.wikipedia.org (parse) Lebanese_Civil_War
en.wikipedia.org (imageinfo) File:Martyrs Square 1982.jpg
Lebanese Civil War (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Martyrs S...
  infobox: <dict(10)> conflict, partof, image, image_size, caption...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:L...
  pageid: 312905
  parsetree: <str(129430)> <root><template><title>short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Lebanese Civil War
  wikibase: Q208484
  wikidata_url: https://www.wikidata.org/wiki/Q208484
  wikitext: <str(107439)> {{short description|1975–1990 civil war ...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Lebanese_Civil_War",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Lebanese Civil War')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
14 July Revolution
17 May Agreement
17 October Revolution
1860 conflict in Lebanon
1935 Yazidi revolt
1935–1936 Iraqi Shia revolts
1943 Barzani revolt
1947–1949 Palestine war
1947–48 Civil War in Mandatory Palestine
1948 Arab–Israeli War
1948 Palestinian exodus
1949 Armistice Agreements
1953 Iranian coup d'état
1958 Lebanon crisis
1959 Mosul uprising
1963 Syrian coup d'état
1964 Hama riot
1966 Syrian coup d'état
1967 Palestinian exodus
1967 Six Day War
1968 Israeli raid on Lebanon
1973 Yom Kippur War
1974–75 Shatt al-Arab conflict
1976 Arab League summit (Riyadh)
1977 Shia uprising in Iraq
1978 South Lebanon conflict
1979 Qatif Uprising
1979–1980 Shia uprising in Iraq
1980 Turkish coup d'état
1982 Beirut bombing
1982 Iranian diplomats kidnapping
1982 Lebanon War
1982 kidnapping of Iranian diplomats
1983 Beirut barracks bombing
1983 Beirut barracks bombings
1983 United States embassy bombing
1984 Sohmor massacre
1984 United States embassy annex bombing
1984 United States embassy 


Page 2
List of speakers of the Parliament of Lebanon
List of wars involving Israel
Litani River
Little Satan
Lod Airport massacre
London Conference of 1939
London Conference of 1946–1947
Ma'alot massacre
Ma'n dynasty
Maarouf Saad
Madrid Conference of 1991
Mafia
Mahmoud Ahmadinejad and Israel
Mahmud Barzanji revolts
Mahsum Korkmaz
Majid Shahriari
Malcolm H. Kerr
Mamluk Sultanate (Cairo)
Mandate for Syria and the Lebanon
Mandatory Palestine
Marada Brigade
Marada Movement
March 14 Alliance
March 2012 Gaza–Israel clashes
March 2017 Israel–Syria incident
March 8 Alliance
Marcia C. Inhorn
Marjayoun
Maronite
Martyrs' Square, Beirut
Marxist-Leninist
Masoud Alimohammadi
May 17 Agreement
May 2013 Rif Dimashq airstrikes
McMahon–Hussein Correspondence
Memory Box (film)
Menachem Begin
Metn
Michel Aoun
Middle Eastern theatre of World War I
Military of Lebanon
Mirdasid dynasty
Misgav Am hostage crisis
Mohammad Hussein Fadlallah
Mohsen Fakhrizadeh
Monte Melkonian
Moral authority
Morocco
Morrison–Grad

In [4]:
def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

def to_delete(s):
    return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def get_list_items(div):
    s = re.sub('{{.*?}}', '', div)
    s = re.sub('\'', '', s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
    s = re.sub('\n----', '<br>', s)
    s = re.sub('\n\*', '<br>', s)
    s = re.sub('<br />', '<br>', s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('</ref>|ref||</ref>', '', s)
    s = re.sub('</ref>', '', s)
    s = re.sub('ref', '', s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     if '<br>' in s:
    s = s.split("<br>")
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
    s = [remove_file_instances(e).strip() for e in s]
    s = [e for e in s if not to_delete(e)]
    s = list(set(s)) # keep unique entries
    return s

In [5]:
df_political_party = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Relegion', 'Founder(s)',
                                          'National Affiliation', 'International Affiliation', 
                                           # 'Allies', 'Opponents',
                                           'Headquarter(s)'])

In [6]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                founders += ''.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    headquarters = get_list_items(page.data['infobox']['headquarters']) if 'headquarters' in page.data['infobox'] else '-'
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    national = get_list_items(page.data['infobox']['national']) if 'national' in page.data['infobox'] else '-'
                    international = get_list_items(page.data['infobox']['international']) if 'international' in page.data['infobox'] else '-'
                    
                    df_political_party = df_political_party.append({
                        'Name': name,
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Relegion': ','.join(religion) if isinstance(religion, list) else religion,
                        'Founder(s)': founders,
                        'National Affiliation': ','.join(national) if isinstance(national, list) else national,
                        'International Affiliation': ','.join(international) if isinstance(international, list) else international,
                        'Headquarter(s)': ','.join(headquarters) if isinstance(headquarters, list) else headquarters
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('headquarters: ', headquarters)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    print('relegion: ', religion)
                    print('national: ', national)
                    print('international: ', international)
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 14 July Revolution
en.wikipedia.org (imageinfo) File:Abd al-Karim Qasim & Abd al-Sal...
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 17 October Revolution
en.wikipedia.org (imageinfo) File:Beirut Protests 2019.jpg
en.wikipedia.org (parse) 1860 conflict in Lebanon
en.wikipedia.org (imageinfo) File:DamasChristianQuarter1860.jpg
en.wikipedia.org (parse) 1935 Yazidi revolt
en.wikipedia.org (parse) 1935–1936 Iraqi Shia revolts
en.wikipedia.org (parse) 1943 Barzani revolt
en.wikipedia.org (parse) 1947–1949 Palestine war
en.wikipedia.org (parse) 1947–48 Civil War in Mandatory Palestine
en.wikipedia.org (imageinfo) File:Katamon.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1953 Iranian coup d'état
en.wikipedia.org (imageinfo) File:Operationajax.jpg
en.

-----------------
name:  ['Fatah – The Revolutionary Council']
founders:  Abu Nidal:
headquarters:  -
ideology:  ['Anti-Zionism', 'Palestinian nationalism', 'Secularism']
position:  -
relegion:  -
national:  -
international:  -


en.wikipedia.org (parse) Abu Nidal Organization internal executions
en.wikipedia.org (parse) Achaemenid Phoenicia
en.wikipedia.org (parse) Aden Emergency
en.wikipedia.org (imageinfo) File:South Yemen in its region.svg
en.wikipedia.org (parse) Administrative divisions of Lebanon
en.wikipedia.org (parse) Adwan Rebellion
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Agriculture in Lebanon
en.wikipedia.org (parse) Ahmed Barzani revolt
en.wikipedia.org (imageinfo) File:3.7inchHowitzerBarzanOperations...
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Akkar
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades


-----------------
name:  ['al-Aqsa Martyrs Brigades', 'Katāib Shuhadā al-Aqṣā']
founders:  Fadi KafishaMarwan Barghouti (convicted)Zakaria Zubeidi (former)Yasser Arafat (former)Naif Abu-Sharah:
headquarters:  -
ideology:  ['Anti-Zionism', 'Palestinian nationalism', 'Socialism']
position:  -
relegion:  -
national:  -
international:  -


en.wikipedia.org (parse) Al-Mourabitoun


...

-----------------
name:  ['Syrian Social Nationalist Party']
founders:  General Secretary:Fares al-Saad,Founder:Antoun Saadeh,
headquarters:  ['Damascus (Syria)', 'Beirut (Lebanon)']
ideology:  ['Economic populism', 'Historical:', 'Antisemitism', 'Fascism (accused,', 'date', 'Pan-Syrianism', 'Anti-Zionism', 'Page needed', 'Social nationalism', 'See: Adel Beshara (ed.), Antun Saadeh: the Man, his thought (2007)', 'Antun Saadeh, The Genesis of Nations, (Dar al-Fikr, Beirut)', 'Syrian irredentism', 'Antun Saadeh, "The Explanation of the Principles". URL: http://www.ssnp.com/new/library/saadeh/principles/', 'August 2021', 'Secularism', 'officially denied)', 'Syrian nationalism']
position:  ['Left-wing']
relegion:  -
national:  ['March 8 Alliance', 'National Progressive Front']
international:  -


en.wikipedia.org (parse) Syrian civil war
en.wikipedia.org (parse) Syrian civil war spillover in Lebanon
en.wikipedia.org (imageinfo) File:Lebanese army on Syria Street.jpg
en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (parse) Syrian occupation of Lebanon
en.wikipedia.org (imageinfo) File:Cedar Revolution Demonstrators.jpg
en.wikipedia.org (parse) Syria–Lebanon campaign
en.wikipedia.org (imageinfo) File:AUSTRALIAN FORCES IN LEBANON, 1...
en.wikipedia.org (parse) Ta'if
en.wikipedia.org (parse) Taba Summit
en.wikipedia.org (parse) Taif Agreement
en.wikipedia.org (parse) Tawhid Movement


-----------------
name:  ['Islamic Unification Movement', 'حركة التوحيد الإسلامي']
founders:  Hashem MinqaraBilal ShaabanSaid Shaaban:
headquarters:  ['Bab al-Tabbaneh (Tripoli)']
ideology:  ['Islamism']
position:  -
relegion:  -
national:  -
international:  -


en.wikipedia.org (parse) Tehran (TV series)
en.wikipedia.org (imageinfo) File:Tehran TV poster.jpg
en.wikipedia.org (parse) Tel al-Zaatar massacre
en.wikipedia.org (parse) Telecommunications in Lebanon
en.wikipedia.org (parse) Television in Lebanon
en.wikipedia.org (parse) Terrorism in Egypt
en.wikipedia.org (parse) Terrorism in Saudi Arabia
en.wikipedia.org (parse) The Clinton Parameters
en.wikipedia.org (parse) The Fateful Triangle: The United States,...
en.wikipedia.org (imageinfo) File:The Fateful Triangle.jpg
en.wikipedia.org (parse) The Human League
en.wikipedia.org (imageinfo) File:The Human League 2007.jpg
en.wikipedia.org (parse) The Lebanon (song)
en.wikipedia.org (imageinfo) File:LebanonHL.jpg
en.wikipedia.org (parse) The Vortices of Wrath (Lebanon 1977)
en.wikipedia.org (parse) Tigers Militia
en.wikipedia.org (parse) Tigers Militia (Lebanon)
en.wikipedia.org (parse) Timeline of Lebanese history
en.wikipedia.org (parse) Timeline of terrorism in Egypt (2013–present)
en.wikipe

skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Manual of Style/Words to watch
skipping Wikipedia:Neutral point of view
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Borders of Lebanon
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Campaignbox Lebanon conflicts
skipping Template:Country data Arab League
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data Libya
skipping Template:Country data PLO
skipping Template:Country data Saudi Arabia
skipping Template:Country data South Yemen
skipping Template:Country data Sudan
skipping Template:Country data Syria
skipping Template:Country data UAE
skipping Template:Country data United Nations
skipping Template:C

In [7]:
df_political_party

,Name,Ideology,Political Position,Relegion,Founder(s),National Affiliation,International Affiliation,Headquarter(s)
0,[Fatah – The Revolutionary Council],"Anti-Zionism,Palestinian nationalism,Secularism",-,-,Abu Nidal:,-,-,-
1,"[al-Aqsa Martyrs Brigades, Katāib Shuhadā al-A...","Anti-Zionism,Palestinian nationalism,Socialism",-,-,Fadi KafishaMarwan Barghouti (convicted)Zakari...,-,-,-
2,[Independent Nasserite Movement (Al-Mourabitoun)],"Arab nationalism,Nasserism,Pan-Arabism,Anti-im...",Left-wing,-,Ibrahim Kulaylat:,-,-,Beirut
3,[Independent Nasserite Movement (Al-Mourabitoun)],"Arab nationalism,Nasserism,Pan-Arabism,Anti-im...",Left-wing,-,Ibrahim Kulaylat:,-,-,Beirut
4,[Amal Movement],"Populism,Conservatism",Centre-right,Shia Islam,"Chairman:Nabih Berri,Founders:Hussein el-Husse...",March 8 Alliance,-,Beirut
5,[Amal Movement],"Populism,Conservatism",Centre-right,Shia Islam,"Chairman:Nabih Berri,Founders:Hussein el-Husse...",March 8 Alliance,-,Beirut
6,[Arab Democratic Party],"Arab nationalism,Baathism,Pan-Syrianism",Left-wing,-,Rifaat Eid:,March 8 Alliance,-,Tripoli
7,[Arab Liberation Army],"Anti-Zionism,Arab nationalism,Pan-Arabism",-,-,Fawzi al-Qawuqji:,-,-,Damascus
8,[Arab Liberation Front],,-,-,Rakad Salem:,PLO,Iraqi-led Baath Party,"Ramallah, Palestine"
9,[Arab Socialist Baath Party – Lebanon Region],Baathism,-,-,Disputed:,March 8 Alliance,Syrian-led Baath Party,Beirut


In [8]:
df_political_party.to_csv('poltical_parties.csv', index=False)

In [9]:
df_politician = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Cause of Death', 'Occupation', 
                                      'Profession', 'Religion', 'Birth Date', 'Birth Place', 'Death Date', 'Death Place'])

In [10]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                    
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    birth_date = get_list_items(page.data['infobox']['birth_date']) if 'birth_date' in page.data['infobox'] else '-'
                    birth_place = get_list_items(page.data['infobox']['birth_place']) if 'birth_place' in page.data['infobox'] else '-'
                    death_date = get_list_items(page.data['infobox']['death_date']) if 'death_date' in page.data['infobox'] else '-'
                    death_place = get_list_items(page.data['infobox']['death_place']) if 'death_place' in page.data['infobox'] else '-'
                    cause_of_death = get_list_items(page.data['infobox']['death_cause']) if 'death_cause' in page.data['infobox'] else '-'
                    profession = get_list_items(page.data['infobox']['profession']) if 'profession' in page.data['infobox'] else '-'
                    occupation = get_list_items(page.data['infobox']['occupation']) if 'occupation' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                    
                    df_politician = df_politician.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Cause of Death': ','.join(cause_of_death) if isinstance(cause_of_death, list) else cause_of_death,
                        'Occupation': ','.join(occupation) if isinstance(occupation, list) else occupation,
                        'Profession': ','.join(profession) if isinstance(profession, list) else profession,
                        'Relegion': ','.join(religion) if isinstance(religion, list) else religion,
                        'Birth Date': ','.join(birth_date) if isinstance(birth_date, list) else birth_date,
                        'Birth Place': ','.join(birth_place) if isinstance(birth_place, list) else birth_place,
                        'Death Date': ','.join(death_date) if isinstance(death_date, list) else death_date,
                        'Death Place': ','.join(death_place) if isinstance(death_place, list) else cause_of_death,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('native_name: ', native_name)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    print('birth_date: ', birth_date)
                    print('birth_place: ', birth_place)
                    print('death_date: ', death_date)
                    print('death_place: ', death_place)
                    print('cause of death: ', cause_of_death)
                    print('occupation: ', occupation)
                    print('profession: ', profession)
                    print('native_name: ', native_name)
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 14 July Revolution
en.wikipedia.org (imageinfo) File:Abd al-Karim Qasim & Abd al-Sal...
en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 17 October Revolution
en.wikipedia.org (imageinfo) File:Beirut Protests 2019.jpg
en.wikipedia.org (parse) 1860 conflict in Lebanon
en.wikipedia.org (imageinfo) File:DamasChristianQuarter1860.jpg
en.wikipedia.org (parse) 1935 Yazidi revolt
en.wikipedia.org (parse) 1935–1936 Iraqi Shia revolts
en.wikipedia.org (parse) 1943 Barzani revolt
en.wikipedia.org (parse) 1947–1949 Palestine war
en.wikipedia.org (parse) 1947–48 Civil War in Mandatory Palestine
en.wikipedia.org (imageinfo) File:Katamon.jpg
en.wikipedia.org (parse) 1948 Arab–Israeli War
en.wikipedia.org (imageinfo) File:Raising the Ink Flag at Umm Ras...
en.wikipedia.org (parse) 1948 Palestinian exodus
en.wikipedia.org (parse) 1949 Armistice Agreements
en.wikipedia.org (parse) 1953 Iranian coup d'état
en.wikipedia.org (imageinfo) File:Operationajax.jpg
en.

-----------------
name:  ['Abbas al-Moussawi']
native_name:  -
nickname:  -
religion:  ['Shia Islam']
native_name:  -
party:  -
nationality:  ['Lebanese']
birth_date:  []
birth_place:  ['Al-Nabi Shayth, Lebanon']
death_date:  []
death_place:  ['Nabatieh Governorate, Lebanon']
cause of death:  ['Assassination']
occupation:  ['Cleric']
profession:  -
native_name:  -


en.wikipedia.org (parse) Abbasid Caliphate
en.wikipedia.org (parse) Abraham Accords
en.wikipedia.org (parse) Abu Hasan (boat)
en.wikipedia.org (parse) Abu Nidal
en.wikipedia.org (imageinfo) File:Abu Nidal.gif


-----------------
name:  ['Abu Nidal']
native_name:  -
nickname:  -
religion:  -
native_name:  -
party:  -
nationality:  ['Palestinian']
birth_date:  ['May 1937']
birth_place:  ['Jaffa, Mandatory Palestine']
death_date:  []
death_place:  ['Baghdad, Baathist Iraq']
cause of death:  -
occupation:  -
profession:  -
native_name:  -


en.wikipedia.org (parse) Abu Nidal Organization
en.wikipedia.org (parse) Abu Nidal Organization internal executions
en.wikipedia.org (parse) Achaemenid Phoenicia
en.wikipedia.org (parse) Aden Emergency
en.wikipedia.org (imageinfo) File:South Yemen in its region.svg
en.wikipedia.org (parse) Administrative divisions of Lebanon
en.wikipedia.org (parse) Adwan Rebellion
en.wikipedia.org (parse) Agreement on Disengagement between Israe...
en.wikipedia.org (parse) Agriculture in Lebanon
en.wikipedia.org (parse) Ahmed Barzani revolt
en.wikipedia.org (imageinfo) File:3.7inchHowitzerBarzanOperations...
en.wikipedia.org (parse) Ain es Saheb airstrike
en.wikipedia.org (imageinfo) File:Ein saheb strike.jpg
en.wikipedia.org (parse) Akkar
en.wikipedia.org (parse) Al-Aqsa Martyrs' Brigades
en.wikipedia.org (parse) Al-Mourabitoun
en.wikipedia.org (parse) Al-Murabitun
en.wikipedia.org (parse) Al-Qaeda insurgency in Yemen
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Al-Wathbah uprising
en.

...

-----------------
name:  ['Yitzhak Navon']
native_name:  -
nickname:  -
religion:  -
native_name:  -
party:  ['Alignment']
nationality:  ['Israeli']
birth_date:  []
birth_place:  ['Jerusalem, British Mandate of Palestine']
death_date:  []
death_place:  ['Jerusalem, Israel']
cause of death:  -
occupation:  -
profession:  ['Author']
native_name:  -


en.wikipedia.org (parse) Yom Kippur War
en.wikipedia.org (imageinfo) File:Yom Kippur War Montage.png
en.wikipedia.org (parse) Zahleh
en.wikipedia.org (parse) Ze'ev Schiff
en.wikipedia.org (parse) Zgharta
en.wikipedia.org (parse) Zghorta
en.wikipedia.org (parse) Zionist entity
en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cropped).jpg


-----------------
name:  ['Émile Lahoud']
native_name:  -
nickname:  -
religion:  -
native_name:  -
party:  ['Independent']
nationality:  -
birth_date:  []
birth_place:  ['Baabdat, Lebanon']
death_date:  -
death_place:  -
cause of death:  -
occupation:  -
profession:  -
native_name:  -


en.wikipedia.org (parse) Talk:Lebanese Civil War


skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Manual of Style/Words to watch
skipping Wikipedia:Neutral point of view
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Borders of Lebanon
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Campaignbox Lebanon conflicts
skipping Template:Country data Arab League
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data Libya
skipping Template:Country data PLO
skipping Template:Country data Saudi Arabia
skipping Template:Country data South Yemen
skipping Template:Country data Sudan
skipping Template:Country data Syria
skipping Template:Country data UAE
skipping Template:Country data United Nations
skipping Template:C

In [11]:
df_politician

,Name,Political Party,Nationality,Cause of Death,Occupation,Profession,Religion,Birth Date,Birth Place,Death Date,Death Place,Relegion
0,"(Abbas al-Moussawi,)",-,Lebanese,Assassination,Cleric,-,NaN,,"Al-Nabi Shayth, Lebanon",,"Nabatieh Governorate, Lebanon",Shia Islam
1,"(Abu Nidal,)",-,Palestinian,-,-,-,NaN,May 1937,"Jaffa, Mandatory Palestine",,"Baghdad, Baathist Iraq",-
2,"(Alfred M. Gray Jr.,)",-,-,-,-,-,NaN,,"Rahway, New Jersey",-,-,-
3,"(Ali Eid,)",Arab Democratic Party,-,-,-,Politician,NaN,,"Tripoli, Lebanon",,"Damascus, Syria",-
4,"(Amine Gemayel,)",Kataeb Party,Lebanese,-,-,-,NaN,,"Bikfaya, Greater Lebanon",-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
87,"(William OCallaghan,)",-,-,-,-,-,NaN,,"Buttevant, County Cork, Ireland",,-,-
88,"(Yasser Arafat,)",Fatah,Palestinian,-,-,Civil engineer,NaN,4 /,"Cairo, Egypt",,"Clamart, Hauts-de-Seine, France",-
89,"(Yasser Arafat,)",Fatah,Palestinian,-,-,Civil engineer,NaN,4 /,"Cairo, Egypt",,"Clamart, Hauts-de-Seine, France",-
90,"(Yitzhak Navon,)",Alignment,Israeli,-,-,Author,NaN,,"Jerusalem, British Mandate of Palestine",,"Jerusalem, Israel",-


In [12]:
df_politician.to_csv('politicians.csv', index=False)

In [13]:
political_events = {}
# max_belligerents = 0
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'perps' in page.data['infobox'] or 'combatant1' in page.data['infobox']:
                    print(page)

                    conflict = get_list_items(page.data['infobox']['conflict']) if 'conflict' in page.data['infobox'] else '-'
                    title = get_list_items(page.data['infobox']['title']) if 'title' in page.data['infobox'] else '-'
                    partof = get_list_items(page.data['infobox']['partof']) if 'partof' in page.data['infobox'] else '-'
                    date = get_list_items(page.data['infobox']['date']) if 'date' in page.data['infobox'] else '-'
                    place = get_list_items(page.data['infobox']['place']) if 'place' in page.data['infobox'] else '-'
                    location = get_list_items(page.data['infobox']['location']) if 'location' in page.data['infobox'] else '-'
                    target = get_list_items(page.data['infobox']['target']) if 'target' in page.data['infobox'] else '-'
                    typec = get_list_items(page.data['infobox']['type']) if 'type' in page.data['infobox'] else '-'
                    perps = get_list_items(page.data['infobox']['perps']) if 'perps' in page.data['infobox'] else '-'
                    
                    if conflict == '-':
                        name = title
                    else:
                        name = conflict if conflict != '-' else date
                    
                    name = ''.join(name)
                    
                    political_events[str(name)] = {
                        'Name': name,
                        'Part of': partof,
                        'Date': date,
                        'Location': place if place != '-' else location,
                        'Target': target,
                        'Type': typec,
                        'Perpetrators': perps
                    }
                    
                    print('-----------------')
                    print('name: ', name)
                    print('conflict: ', conflict)
                    print('title: ', title)
                    print('partof: ', partof)
                    print('date: ', date)
                    print('place: ', place)
                    print('location: ', location)
                    print('target: ', target)
                    print('type: ', typec)
                    print('perps: ', perps)
                    
                    if 'combatant1' in page.data['infobox']:
                        keys = list(page.data['infobox'].keys())
                        belligerents_commanders = [e for e in keys if 'combatant' in e or 'commander' in e]
                        for k in belligerents_commanders:
                            v = get_list_items(page.data['infobox'][k]) if k in page.data['infobox'] else '-'
                            political_events[str(name)][k] = v
                            print('{}: {}'.format(k, v))
#                         size = len(belligerents_commanders)/2
#                         max_belligerents = size if size > max_belligerents else max_belligerents
                        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 14 July Revolution
en.wikipedia.org (imageinfo) File:Abd al-Karim Qasim & Abd al-Sal...


-----------------
name:  14 July Revolution
conflict:  ['14 July Revolution']
title:  -
partof:  ['the Arab Cold War']
date:  ['14 July 1958']
place:  ['Iraq']
location:  -
target:  -
type:  -
perps:  -
combatant1: ['**  Royal Guard\nSupported by:', 'Arab Federation']
combatant2: ['Free Officers', '20th Brigade\nSupported by:', '19th Brigade']
commander1: ['Nuri al-Said', 'King Faisal&nbsp;II', 'Abd al-Ilah']
commander2: ['Nazem Tabakli', 'Abdul Salam Arif', 'Surat al-Haj Sri', 'Abd al-Karim Qasim', 'Muhammad Najib ar-Rubai']


en.wikipedia.org (parse) 17 May Agreement
en.wikipedia.org (parse) 17 October Revolution
en.wikipedia.org (imageinfo) File:Beirut Protests 2019.jpg
en.wikipedia.org (parse) 1860 conflict in Lebanon
en.wikipedia.org (imageinfo) File:DamasChristianQuarter1860.jpg


-----------------
name:  1860 civil conflict in Mount Lebanon and Damascus
conflict:  ['1860 civil conflict in Mount Lebanon and Damascus']
title:  -
partof:  -
date:  ['23 May – 11 July 1860']
place:  ['Mount Lebanon, Beqaa Valley, Galilee and Damascus, Ottoman Empire']
location:  -
target:  -
type:  -
perps:  -
combatant1: ['Rural Maronite militiamen', 'Shihab dynasty\nSupported by', 'Zahalni militiamen', 'Maronites and allies', 'French expeditionary forces']
combatant2: ['al-Atrash clan', 'Talhuq clan', 'Abu Nakad clan', 'Jumblatt clan\nSupported by', 'Harfush clan', 'Rural Druze clans', 'Imad clan', 'Rural Sunni and Shia Muslim militiamen', 'Sardiyah tribe']
commander1: ['Yusuf al-Mubayyad', 'Yusuf al-Shantiri', 'Abdallah Abu Khatir', 'Youssef Bey Karam', 'Tanyus Shahin']
commander2: ['Ali Imad', 'Khattar Imad', 'Said Jumblatt', 'Kenj Ahmad', 'Hasan Agha al-Tawil', 'KIA', 'Bashir Nakad', 'Ismail al-Atrash', 'Qasim Imad']


en.wikipedia.org (parse) 1935 Yazidi revolt
en.wikipedia.org (parse) 1935–1936 Iraqi Shia revolts
en.wikipedia.org (parse) 1943 Barzani revolt
en.wikipedia.org (parse) 1947–1949 Palestine war


...

-----------------
name:  Second Yemenite War
conflict:  ['Second Yemenite War']
title:  -
partof:  ['the Cold War, the Sino-Soviet split, and the Arab Cold War']
date:  ['24 February – 19 March 1979', '(  )']
place:  ['North Yemen-South Yemen border']
location:  -
target:  -
type:  -
perps:  -
combatant1: ['Taiwan', 'Egypt', 'United States', 'Supported by:', 'Flag', 'Flagicon image', 'Saudi Arabia', 'United Kingdom', 'Jordan', '1972']
combatant2: ['Cuba', 'Libya', 'East Germany', 'Supported by:', '1948', 'Flag', 'North Korea', 'Soviet Union', 'Flagicon image']
commander1: ['Ali Abdullah Saleh']
commander2: ['Abdul Fattah Ismail']


en.wikipedia.org (parse) Yemeni–Adenese clan violence
en.wikipedia.org (parse) Yezid Sayigh
en.wikipedia.org (parse) Yitzhak Navon
en.wikipedia.org (imageinfo) File:Portrait of MP Yitzhak Navon.jpg
en.wikipedia.org (parse) Yom Kippur War
en.wikipedia.org (imageinfo) File:Yom Kippur War Montage.png


-----------------
name:  Fourth Arab–Israeli War
conflict:  ['Fourth Arab–Israeli War']
title:  -
partof:  ['the Arab–Israeli conflict and the Cold War']
date:  ['6–25 October 1973', '(  )']
place:  ['Golan Heights, Sinai Peninsula, Suez Canal (both banks) and surrounding regions']
location:  -
target:  -
type:  -
perps:  -
combatant1: ['Supported by:']
combatant2: ['Cuba', 'Libya', 'Tunisia', 'Egypt', 'Expeditionary forces:', 'Morocco', 'Algeria', 'Kuwait', 'link', 'Syria', 'Perez, Cuba, Between Reform and Revolution, pp. 377–79. Gott, Cuba, A New History, p. 280.', 'Iraq  Libya', 'flagicon image', 'Iraq', 'Jordan']
commander1: ['Albert Mandler', 'Israel Tal', 'Ariel Sharon', 'David Elazar', 'Golda Meir', 'Binyamin Peled', 'Albert Mandler  *  Ariel Sharon', 'Benjamin Telem', 'Shmuel Gonen', 'Moshe Dayan', 'KIA', 'Yitzhak Hofi', 'Israel', 'Haim Bar-Lev']
commander2: ['Omar Abrash', 'Hafez al-Assad', 'Egypt', 'Ahmad Ismail Ali', 'Anwar Sadat', 'Saad El Shazly', 'Abdel Ghani el-Gammasy',

en.wikipedia.org (parse) Zahleh
en.wikipedia.org (parse) Ze'ev Schiff
en.wikipedia.org (parse) Zgharta
en.wikipedia.org (parse) Zghorta
en.wikipedia.org (parse) Zionist entity
en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cropped).jpg
en.wikipedia.org (parse) Talk:Lebanese Civil War


skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Manual of Style/Words to watch
skipping Wikipedia:Neutral point of view
skipping Wikipedia:Protection policy
skipping Wikipedia:Verifiability
skipping Template:Arab–Israeli armed engagements
skipping Template:Arab–Israeli conflict
skipping Template:Arab–Israeli conflict participants
skipping Template:Arab–Israeli diplomacy
skipping Template:Borders of Lebanon
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Campaignbox Lebanon conflicts
skipping Template:Country data Arab League
skipping Template:Country data Israel
skipping Template:Country data Lebanon
skipping Template:Country data Libya
skipping Template:Country data PLO
skipping Template:Country data Saudi Arabia
skipping Template:Country data South Yemen
skipping Template:Country data Sudan
skipping Template:Country data Syria
skipping Template:Country data UAE
skipping Template:Country data United Nations
skipping Template:C

In [14]:
# max_belligerents

In [15]:
political_events

{'14 July Revolution': {'Name': '14 July Revolution',
  'Part of': ['the Arab Cold War'],
  'Date': ['14 July 1958'],
  'Location': ['Iraq'],
  'Target': '-',
  'Type': '-',
  'Perpetrators': '-',
  'combatant1': ['**  Royal Guard\nSupported by:', 'Arab Federation'],
  'combatant2': ['Free Officers',
   '20th Brigade\nSupported by:',
   '19th Brigade'],
  'commander1': ['Nuri al-Said', 'King Faisal&nbsp;II', 'Abd al-Ilah'],
  'commander2': ['Nazem Tabakli',
   'Abdul Salam Arif',
   'Surat al-Haj Sri',
   'Abd al-Karim Qasim',
   'Muhammad Najib ar-Rubai']},
 '1860 civil conflict in Mount Lebanon and Damascus': {'Name': '1860 civil conflict in Mount Lebanon and Damascus',
  'Part of': '-',
  'Date': ['23 May – 11 July 1860'],
  'Location': ['Mount Lebanon, Beqaa Valley, Galilee and Damascus, Ottoman Empire'],
  'Target': '-',
  'Type': '-',
  'Perpetrators': '-',
  'combatant1': ['Rural Maronite militiamen',
   'Shihab dynasty\nSupported by',
   'Zahalni militiamen',
   'Maronites and 

In [16]:
df_political_event = pd.DataFrame(columns=['Name', 'Location', 'Date', 'Target', 'Type', 'Perpetrators',
                                           'Combatant1', 'Combatant2', 'Combatant3', 'Combatant4',
                                           'Commander1', 'Commander2', 'Commander3', 'Commander4',
                                           'Part of'])

In [17]:
for k in political_events:
#     print(political_events[k]['Name'])
    conflict =   political_events[k]['Name']
    partof =     political_events[k]['Part of']
    date =       political_events[k]['Date']
    location =   political_events[k]['Location']
    target =     political_events[k]['Target']
    typec =      political_events[k]['Type']
    perps =      political_events[k]['Perpetrators']
    combatant1 = political_events[k]['combatant1'] if 'combatant1' in political_events[k] else '-'
    combatant2 = political_events[k]['combatant2'] if 'combatant2' in political_events[k] else '-'
    combatant3 = political_events[k]['combatant3'] if 'combatant3' in political_events[k] else '-'
    combatant4 = political_events[k]['combatant4'] if 'combatant4' in political_events[k] else '-'
    
    commander1 = political_events[k]['commander1'] if 'commander1' in political_events[k] else '-'
    commander2 = political_events[k]['commander2'] if 'commander2' in political_events[k] else '-'
    commander3 = political_events[k]['commander3'] if 'commander3' in political_events[k] else '-'
    commander4 = political_events[k]['commander4'] if 'commander4' in political_events[k] else '-'
    
    df_political_event = df_political_event.append({
                            'Name':         ','.join(conflict) if isinstance(conflict, list) else conflict,
                            'Location':     ','.join(location) if isinstance(location, list) else location,
                            'Date':         ','.join(date) if isinstance(date, list) else date,
                            'Target':       ','.join(target) if isinstance(target, list) else target,
                            'Type':         ','.join(typec) if isinstance(typec, list) else typec,
                            'Perpetrators': ','.join(perps) if isinstance(perps, list) else perps,
                            'Combatant1':   ','.join(combatant1) if isinstance(combatant1, list) else combatant1,
                            'Combatant2':   ','.join(combatant2) if isinstance(combatant1, list) else combatant2,
                            'Combatant3':   ','.join(combatant3) if isinstance(combatant1, list) else combatant3,
                            'Combatant4':   ','.join(combatant4) if isinstance(combatant1, list) else combatant4,
                            
                            'Commander1':   ','.join(commander1) if isinstance(commander1, list) else commander1,
                            'Commander2':   ','.join(commander2) if isinstance(commander2, list) else commander2,
                            'Commander3':   ','.join(commander3) if isinstance(commander3, list) else commander3,
                            'Commander4':   ','.join(commander4) if isinstance(commander4, list) else commander4,
                            
                            'Part of':      ','.join(partof) if isinstance(partof, list) else partof,
                        }, ignore_index=True)

In [18]:
df_political_event

,Name,Location,Date,Target,Type,Perpetrators,Combatant1,Combatant2,Combatant3,Combatant4,Commander1,Commander2,Commander3,Commander4,Part of
0,14 July Revolution,Iraq,14 July 1958,-,-,-,"** Royal Guard\nSupported by:,Arab Federation","Free Officers,20th Brigade\nSupported by:,19th...",-,-,"Nuri al-Said,King Faisal&nbsp;II,Abd al-Ilah","Nazem Tabakli,Abdul Salam Arif,Surat al-Haj Sr...",-,-,the Arab Cold War
1,1860 civil conflict in Mount Lebanon and Damascus,"Mount Lebanon, Beqaa Valley, Galilee and Damas...",23 May – 11 July 1860,-,-,-,"Rural Maronite militiamen,Shihab dynasty\nSupp...","al-Atrash clan,Talhuq clan,Abu Nakad clan,Jumb...",-,-,"Yusuf al-Mubayyad,Yusuf al-Shantiri,Abdallah A...","Ali Imad,Khattar Imad,Said Jumblatt,Kenj Ahmad...",-,-,-
2,1947–1949 Palestine war,"Former Mandatory Palestine, Sinai Peninsula, s...","( ),30 November 1947&nbsp;– 20 July 1949",-,-,-,"border]] Minorities Unit,Yishuv,----\nBefore 2...","* Egypt,----\n :,* All-Palestine Protectora...",-,-,"Yigal Allon,Yisrael Galili,Yaakov Dori,David S...","Haj Amin Al-Husseini,Habis al-Majali,Ahmad Ali...",-,-,the Sectarian conflict in Mandatory Palestine ...
3,Civil war in Palestine (1947–48),Mandatory Palestine,"30 November 1947 – 14 May 1948,( )",-,-,-,"Lehi,Jews of Palestine,Foreign Volunteers,Alli...","Arabs of Palestine,Army of the Holy War",* United Kingdom military forces in Mandatory...,-,"Yigal Allon,Yaakov Dori,David Ben-Gurion,Menac...","Abd al-Qadir al-Husayni,Fawzi al-Qawuqji",Gordon MacMillan,-,the Intercommunal conflict in Mandatory Palest...
4,1948 Arab–Israeli War,"Former British Mandate of Palestine, Sinai Pen...","15 May 1948 – 10 March 1949,( )",-,-,-,"border]] Minorities Unit,Yishuv,Before 26 May ...",** All-Palestine Protectorate *** Holy War ...,-,-,"Yigal Allon,Mickey Marcus,Yisrael Galili,Yaako...","Habis Majali,Muzahim al-Pachachi,Husni al-Zaim...",-,-,the 1947–1949 Palestine war
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,The War over Water,Jordan Valley River Basin and the Golan Heights,November 1964 – May 1967,-,-,-,,,-,-,"Zalman Shazar,Levi Eshkol,Yitzhak Rabin","Charles Helou,Nureddin al-Atassi,Hussein,Amin ...",-,-,the Arab-Israeli conflict
153,World War I,"Europe, Africa, the Middle East, the Pacific I...","4,Treaty of Saint-Germain-en-Laye,The United S...",-,-,-,"British Empire,(from 1917),United States,Kingd...","flagcountry,Kingdom of Bulgaria,German Empire,...",-,-,"[[Allied leaders of World War I,David Lloyd Ge...","German Empire,Franz Joseph I,and others,Died i...",-,-,-
154,World War&nbsp;II,"Europe, Pacific, Atlantic, Indian Ocean, South...","Age in years and days,1,1 September 1939,(,sta...",-,-,-,Allies,Axis,-,-,"1936,size,Chiang Kai-shek,United States,Winsto...","size,Benito Mussolini,Fascist Italy (1922–1943...",-,-,-
155,Yemeni Civil War,"Yemen, southern Saudi Arabia (spillovers)","),16 September 2014 – present",-,-,-,"* Pro-Saleh forces (until 2017),1,5,2,Syrian ...","4,Bahrain,Italy,United States,Somalia,(2015–16...","1,date,border]] Al-Qaeda,IS-YP,size,Ansar al-S...",-,"4,Ahmad Al-Khatib,Ali Aldhifeef,Ali Al-Jaifi,1...","4,Macky Sall,15,13,Col. Hassan Ageeli,Under 1,...","KIA,Qasim al-Raymi,Nasir al-Wuhayshi,Khalid Ba...",-,"the Yemeni Crisis, Arab Winter, War on terror,..."


In [19]:
df_political_event.to_csv('poltical_events.csv', index=False)